The steps are mostly based on these refrences:
- [Fine-tuning XLS-R for Multi-Lingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2)
- [Fine_Tune_XLSR_Wav2Vec2_on_Persian_ShEMO](https://github.com/m3hrdadfi/notebooks/blob/main/Fine_Tune_XLSR_Wav2Vec2_on_Persian_ShEMO_ASR_with_%F0%9F%A4%97_Transformers_ipynb.ipynb)

You can consult them if you want to see a more detailed procedure.

The second one also contains valuable hints on how to preprocess the persian text for our purpose.

In [ ]:
# Download nessecary libraries
!pip install datasets==2.10.0 --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00


## Loading the dataset

In [ ]:
# If you wish your data to persist even when you shutdown colab, save your intermediate results to your google drive
# Then in the code you can change the saving path of files to ./drive/MyDrive/ path, which is your google drive disk
from os import path,system,mkdir
from google.colab import drive

drive.mount('/content/drive/')
if not path.exists('./drive/MyDrive/ASR_Colab'):
  mkdir('./drive/MyDrive/ASR_Colab')

Mounted at /content/drive/


In [ ]:
# modify the path to dataset if nessecary
dataet_path = './drive/MyDrive/ML_Project_ASR/dataset.zip'
if not path.exists('dataset'):
  system(f'unzip -qq "{dataet_path}" -d "/content/"')

In [ ]:
# Load csv
# We use pandas for data import, and datasets lib to prepear our data.
# These two libs (pandas.Dataframe & datasets.Dataset) are convertable as shown bellow, so use whichever you find more convenient
import pandas as pd
from datasets import Dataset

transcripts = pd.read_csv('Dataset/transcripts.csv')
ds = Dataset.from_pandas(transcripts)
transcripts.head()

,voice_filename,transcript,accent,gender,tone
0,voice_1.mp3,چرا این‌‌‌‌طور فکر می‌‌‌‌کنی؟,فارسی,male,question
1,voice_2.mp3,همیشه من و تو راجع به آن با هم صحبت کرده‌‌‌‌ایم,فارسی,male,normal
2,voice_3.mp3,دنیا در حال گذار به‌‌‌‌سمت پایداری است,فارسی,male,normal
3,voice_4.mp3,شاخصی که باید عملکرد تسلا را با آن اندازه بگیریم,فارسی,male,normal
4,voice_5.mp3,باید تعداد واقعاً غیرقابل‌‌‌‌تصوری باتری تولید...,فارسی,male,normal


In [ ]:
# Take a look at unique letters in our dataset
from functools import reduce
present_chars = reduce(lambda a, b: set((*a,*b)), list(ds['transcript']))
print(present_chars)

{'َ', 'ـ', 'ْ', 'ﻣ', 'ۀ', '٪', 'ﮑ', '\n', 'ً', 'ﻪ', 'ﺮ', '۱', 'غ', 'ض', 'ن', '5', '0', 'ذ', 'آ', 'ﺎ', '٬', '…', 'ﮏ', 'ﻌ', 'ﮔ', 'ه', 'ﻢ', 'ص', '٫', 'ﻩ', 'ﺗ', '!', 'ﺣ', 'ث', ':', 'ا', '-', 'ﺭ', '4', 'S', 'ﺷ', 'ﺲ', 'ﯼ', 'ﻧ', 'ﻓ', 'ﻃ', 'ِ', 'ﻤ', '1', 'ٔ', 'ﺘ', '.', '"', 'ﺑ', 'ﻏ', 'ﺁ', '۹', ',', 'چ', 'ق', 'ئ', 'ﮐ', 'ﺫ', 'ج', '2', 'ﯾ', 'ﭽ', 'ي', 'ﺪ', ')', 'ﻨ', 'ﺼ', 'ﺶ', '9', 'ح', 'ﺟ', 'M', 'ﻮ', 'ژ', 'ﺖ', 'ﻒ', 'ط', 'ف', '۲', 'ز', '۳', '۵', '\xad', 'ى', 'ﺕ', '(', 'ك', 'ﺩ', 'ﻔ', '8', '؟', 'م', 'ؤ', 'ﺯ', 'ﻡ', 'ء', 'ﺳ', 'س', 'ب', 'ت', 'ﯽ', 'ﺛ', '\u200c', 'ش', '”', 'خ', 'ﻬ', '\u202b', 'ی', 'ﺤ', 'V', 'أ', '«', ' ', 'ظ', 'د', '٨', 'ﻫ', 'ر', 'ﭘ', 'و', 'گ', 'ﺴ', '٥', 'ﻠ', '–', 'ﻭ', 'ﺍ', '؛', 'ﻝ', 'ﺥ', '3', '/', 'ﻖ', '6', 'پ', '۷', 'ع', '،', '\t', 'ﯿ', 'ٍ', 'ﻥ', 'ک', '۶', '\u202c', '“', 'ل', 'ﺨ', '7', '۴', '»', 'ﻦ', '\xa0', '۰', 'ُ', 'ّ', '۸'}


## Preprocessing text & audio

In [ ]:
# Some of the listed chars are the same, but have different representations(like 'ب' & 'ﺑ')
# They should get combined(one of them gets mapped to the other)
# complete the following dict:
import re

char_mappings = {'ﺁ':'ا', 'آ':'ا', "ﺎ":"ا", 'ﺍ':"ا", 'أ':'ا', 'بِ':'ب', 'ﺒ':"ب", "ﺑ":"ب", "ﭘ":"پ", 'ﺖ':"ت", 'ﺕ':'ت', 'ﺗ':'ت', "ﺘ":"ت", 'ت':'ت',
                'ﺛ':'ث', 'ﺟ':'ج', 'چ':'چ', 'ﭽ':'چ', 'ﺣ':'ح', 'ﺤ':'ح', "ﺧ":"خ", 'ﺨ':'خ', 'خ':'خ', 'ﺥ':'خ',
                'دِ':'د', "ﺩ":"د", 'ﺪ':"د", 'ذِ':'ذ', 'ﺮ':"ر", 'ﺭ':"ر", 'زِ':'ز', 'ﺯ':'ز',
                'ﺳ':'س', 'ﺲ':'س', "ﺱ":"س", 'سِ':'س', 'ﺴ':"س", 'ﺶ':'ش', 'ﺷ':"ش", 'شِ':'ش', 'ﺸ':"ش", 'ﺼ':'ص',
                "ﻀ":"ض", "ﻌ":"ع", 'ﻋ':"ع", 'ﻏ':'غ', 'ﻔ':'ف', 'ﻓ':'ف', 'ﻒ':'ف', 'ﻖ':'ق', 'ﮏ':'ک', 'ﮑ':'ک', "ﮐ":"ک", 'ك':'ک', "ﮔ":"گ",
                "ﻟ":"ل", 'ﻠ':'ل', 'ﻝ':'ل', "ﻡ":"م", 'ﻣ':'م', "ﻢ":"م", 'ﻤ':"م",'ﻦ':'ن', 'ﻨ':'ن', 'ﻥ':"ن", 'ﻧ':"ن",
                'ؤ':'و', 'ﻭ':"و", "ﻮ":"و", "ﻪ":"ه", 'ﻬ':'ه', "ۀ":"ه", "ۀ":"ه", '٥':'ه', 'ة':"ه", 'ﻩ':'ه', 'ﻫ':'ه',
                "ﯽ":"ی", 'ى':'ی', 'ي':'ی', "ے":"ی", 'ﯼ':'ی', 'ﯾ':"ی", 'ﯿ':"ی", 'ئ':'ی',
                # '۱':'1','۲':'2', '۳':'3', '۴':'4', '۵':'5', '۶':'6', '۷':'7', '۸':'8', '٨':'8', '۹':'9'
                }

def multiple_replace(batch, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    batch['transcript'] = re.sub(pattern, lambda m: chars_to_mapping[m.group()], batch['transcript'])
    return batch

ds = ds.map(lambda batch: multiple_replace(batch,char_mappings))

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
# Some chars don't have any sound, so they should get removed
# Don't remove the ' ' (space) though, as the model should learn to predict when each word ends
# handle the transcripts containing numbers as you deem nessecary
# complete the following list:
import string

char_removals = ['ِ', '(', ')', '!', '-', '،', '«', '…', 'ٍ', '\n', 'ـ', '٫' ,'ً' , '٬', '/', '-', '»',
                 '.', 'َ', 'ّ', ',', '“', '”', '؟', 'ْ', '!', '"', '؛', 'ٔ', '–', ':', 'ُ', '٪', 'ء',
                 '\t', '\u202c', '\u202b', '\u200c','\xad', '\xa0'] + list(string.ascii_letters + string.digits)

def remove_special_characters(batch,char_removals):
    chars_to_ignore_regex = f"""[{"".join(char_removals)}]"""
    batch['transcript'] = re.sub(chars_to_ignore_regex, '', batch['transcript']).lower() + " "
    return batch

ds = ds.map(lambda batch: remove_special_characters(batch,char_removals))

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
# The resulting vocab(list of letters):
vocab = reduce(lambda a, b: set((*a,*b)), ds['transcript'])
print(vocab)

{'ط', 'ف', '۲', 'ز', ' ', 'ظ', '۳', '۵', 'د', '٨', 'ر', 'ﻃ', 'و', 'گ', 'م', 'غ', '۱', 'ض', '۹', 'ن', 'چ', 'ق', 'ذ', 'س', 'ب', 'پ', '۷', 'ت', 'ع', 'ه', 'ﺫ', 'ک', 'ج', 'ص', 'ش', '۶', 'خ', 'ی', 'ل', '۴', 'ث', 'ا', 'ح', '۰', 'ژ', '۸'}


In [ ]:
# Wav2Vec requires some special tokens to be added to vocab
# We also replace ' '(space) with '|' for more visibility
# The vocab should get saved as a json file and later get used by the model
vocab_dict = {v: k for k, v in enumerate(vocab)}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

import json
with open('./drive/MyDrive/ASR_Colab/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

To know what is the role of tokenizer, feature extractor, data collator & etc. in this model, visit https://huggingface.co/blog/fine-tune-xlsr-wav2vec2

In [ ]:
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2FeatureExtractor,Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./drive/MyDrive/ASR_Colab/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
# Tokenizing the transcripts and then load,convert to mono channel and resample audio files at 16 KHz
import librosa
import warnings

def prepare_dataset(batch):
  file_path = path.join('Dataset','voices',batch['voice_filename'])
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    speech_array, sampling_rate = librosa.load(file_path,mono=True,sr=16000)

    batch["input_values"] = processor(speech_array, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids

  return batch

ds = ds.map(prepare_dataset)

Map:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
# To reduce GPU memory usage, filter out voice samples that are too long:
max_input_length_in_sec = 15
ds = ds.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/6042 [00:00<?, ? examples/s]

In [ ]:
ds = ds.train_test_split(0.2)

# A report on dataset length:
ds

DatasetDict({
    train: Dataset({
        features: ['voice_filename', 'transcript', 'accent', 'gender', 'tone', 'input_values', 'input_length', 'labels'],
        num_rows: 4612
    })
    test: Dataset({
        features: ['voice_filename', 'transcript', 'accent', 'gender', 'tone', 'input_values', 'input_length', 'labels'],
        num_rows: 1153
    })
})

In [ ]:
# Save for later use
ds.save_to_disk("./drive/MyDrive/ASR_Colab/dataset")

Flattening the indices:   0%|          | 0/4612 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/4612 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1153 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1153 [00:00<?, ? examples/s]